# 思路
## 推荐算法
### 1 数据
1、左右结构的数据；\
2、上下级数据缺失的数据补全，下级完全不一样\
3、展示数据互为推荐，展示数据的重复推荐\
4、处理掉互为推荐，重复推荐可以不处理
### 2 节点
1、chain(*df.values)，构建节点数据 \
2、节点无下级的数据和有推荐下级的数据分别处理
### 3 构建全层级df
1、节点构建df
2、构建所有下级
### 4 统计发展下限和人数
1、分组统计每个节点的发展下限发展人数\
注意：nan的处理方式
### 5 未发展的节点合并上

In [1]:
import numpy as np
import pandas as pd
from itertools import chain

## 1 数据导入与处理

In [2]:
data = pd.read_excel(r"数据.xlsx",dtype=np.str,encode='utf-8')

In [276]:
data1 = data.copy()
data1.rename(columns={"ID":"userid","邀请人ID":"t_userid"},inplace=True)  # 字段名称统一   userid : 直接下线名称，t_userid : 直接上线名称
data1.head()

,userid,t_userid
0,177618,162700
1,177617,43946
2,177616,43946
3,177615,74898
4,177614,43946


In [277]:
data1=data1.append({"userid":"a","t_userid":"a"},ignore_index=True)

In [278]:
# 数据去重
data1.drop_duplicates(inplace=True)

In [279]:
# 数据缺失处理
# 直接下线缺失填充上不同的值 wz_i(i=0,1,2,3...) ; 直接上线缺失填充上相同的值 wz_s
data1.loc[data1.userid.isnull(),"userid"] = ["wz_"+str(i) for i in range(len(data1[data1.userid.isnull()]))]
data1.fillna("wz_s",inplace=True)

In [288]:
#data1 = data1.append({"userid":"a","t_userid":"b"},ignore_index=True)
#data1 = data1.append({"userid":"b","t_userid":"a"},ignore_index=True)

In [293]:
# 相互推荐数据查找并删除
data_hu_tui = pd.merge(data1,data1,how="left",left_on="t_userid",right_on="userid")
data_hu_tui_1 = data_hu_tui[data_hu_tui.userid_x==data_hu_tui.t_userid_y]         # 相互推荐数据
list_hutui_1 = []
for i in data_hu_tui_1.index:                                                   # 相互推荐的组合    
    list_hutui_1.append([data_hu_tui_1.loc[i][0],data_hu_tui_1.loc[i][1]]) # [userid,t_userid]

list_hutui_1 = list(set([tuple(set(i)) for i in list_hutui_1]))        
list_hutui_2 = []
for j in list_hutui_1:                                   # 找到相互推荐数据（成对出现）的一条，确定在下线出现多次的相互推荐数据
    if len(j)==1:
        list_hutui_2.append(data1[(data1.userid==j[0])&(data1.t_userid==j[0])])     # 自身推荐自身的节点
    else:
        if data1.userid.tolist().count(j[0])>=2:             # A=j[0],B=j[1] 有 A 在下线的数量大于等于2则 A 有除 B 以外的其他上线 删除 B 推荐 A
            list_hutui_2.append(data1[(data1.userid==j[0])&(data1.t_userid==j[1])])
        else:
            list_hutui_2.append(data1[(data1.userid==j[1])&(data1.t_userid==j[0])])
if list_hutui_2 != []:
    list_hutui_3 = pd.concat(list_hutui_2,sort=False).index
    data1.drop(index=list_hutui_3,inplace=True)


In [294]:
data_hu_tui_1

,userid_x,t_userid_x,userid_y,t_userid_y


## 2数据节点

In [256]:
### 所有节点数据
# 列表形式节点
# 列表形式节点
jiedian_data = list(set(list(chain(*data1.values))))
print(jiedian_data[0:5])
print("节点长度：",len(jiedian_data))

['150379', '138233', '64642', '105271', '90352']
节点长度： 124042


### 有推荐的节点和无推荐的节点分离
#### 无推荐的节点不进行计算，缩小数据量

In [260]:
node_df = pd.merge(data1,data1,how="left",left_on="userid",right_on="t_userid",suffixes=("","_y"))
node_df.head()

,userid,t_userid,userid_y,t_userid_y
0,177618,162700,NaN,NaN
1,177617,43946,NaN,NaN
2,177616,43946,NaN,NaN
3,177615,74898,NaN,NaN
4,177614,43946,NaN,NaN


In [261]:
# 没有推荐下线数据 ：userid 不在 t_userid 中的数据
node_1 = node_df[node_df.userid_y.isnull()]
list_node_1 = np.unique(node_1.userid.tolist())
list_node_1

array(['100000', '100001', '100002', ..., '99997', '99998', '99999'],
      dtype='<U6')

In [262]:
# 有推荐下线的节点
list_node_2 = np.unique(node_df.t_userid.tolist())

In [263]:
len(list_node_2)

5145

In [265]:
data1.head()

,userid,t_userid
0,177618,162700
1,177617,43946
2,177616,43946
3,177615,74898
4,177614,43946


In [266]:
# 有推荐的节点与调整数据整合
data_tuijian = pd.DataFrame({"node":list_node_2})
j = -1
i = 0
while j != 0:   
    i +=1
    data_tuijian = pd.merge(data_tuijian,data1,how="left",
                            left_on=data_tuijian.columns[-1],
                            right_on=data1.t_userid,suffixes=("","_%s" % i),)
    data_tuijian.drop(columns=["t_userid"],inplace = True)
    
    j = len(data_tuijian[data_tuijian.columns[-1]].value_counts())
    #print("最大层级":i)

In [268]:
%%time
##  统计发展下线深度和人数

#fuzhu = []         # 辅助任务 1

over = pd.DataFrame(columns=["node","下限","count"])
for name,group_i in data_tuijian.groupby("node"): 
    print(name)                          # 节点    name :节点 ； group ：与name对应的子树
    group_j = group_i.dropna(axis=1,how="all")
    xia = np.unique(list(chain(*group_j.values)))
    xxx = list(filter(lambda x : x!="nan" ,xia))
    c = len(xxx)-1                                # 数量
    #print(c)
    #print(j,y,c)     
    over = over.append({"node":name,"下限":len(group_j.columns)-1,"count":c},ignore_index=True)
    
    # 小批量测试
   # 测试用的 可删除
      #  break

100012
100050
100110
100186
100216
100219
100270
100378
100465
100500
100509
100511
100574
100600
100758
100828
100953
100964
100990
100992
100994
101034
101061
101066
101083
101160
101162
101173
101186
101221
101232
101242
101256
101293
101312
101315
101337
101485
101550
101552
101576
101638
101700
101753
101764
101860
101888
101939
102782
102817
102901
102905
103032
103077
103102
103313
103324
103328
103343
103391
103406
103415
103416
103432
103445
103475
103481
103482
103503
103505
103545
103568
103861
104146
104447
104545
104571
104619
104622
104662
104688
104695
104699
105739
106469
106879
106894
107049
107081
107113
107157
107420
107532
107599
107616
107661
108220
108613
108969
108980
109056
109059
109176
109444
109585
109707
110233
110428
110846
110896
111991
112189
114094
114443
114778
114805
114841
115399
115401
115426
115525
115933
116057
116129
117125
121682
123075
123183
123232
123284
123376
123448
123510
123513
123555
123647
123697
123729
123810
123831
123843
123911
123986

163613
163631
163734
163766
163768
163822
163840
163858
163864
163873
163879
163902
163903
163937
163961
163964
163965
163975
164018
164027
164035
164038
164048
164072
164107
164109
164114
164141
164188
164224
164227
164248
164277
164291
164356
164458
164487
164515
164534
164540
164542
164548
164560
164561
164567
164571
164585
164606
164621
164677
164686
164694
164699
164706
164716
164757
164762
164774
164831
164855
164860
164867
164868
164879
164913
164928
164932
164934
164938
164940
164994
165010
165019
165174
165185
165240
165248
165249
165252
165264
165282
165308
165309
165312
165315
165317
165320
165329
165337
165340
165351
165364
165367
165376
165377
165383
165388
165393
165401
165409
165412
165415
165426
165444
165450
165454
165480
165512
165523
165580
165647
165686
165706
165707
165731
165757
165758
165760
165792
165834
165846
165852
165859
165863
165870
165882
165888
165911
165912
165913
165927
165931
165938
165948
165962
165971
166007
166033
166036
166056
166094
166099
166101

52036
52049
52050
52055
52058
52063
52067
52074
52086
52095
52099
52103
52108
52110
52113
52123
52128
52130
52131
52135
52137
52138
52146
52147
52149
52151
52152
52154
52174
52178
52185
52204
52205
52207
52218
52232
52243
52248
52250
52266
52269
52301
52326
52385
52389
52399
52412
52436
52451
52472
52479
52482
52483
52484
52491
52501
52505
52508
52510
52524
52526
52540
52545
52555
52567
52574
52575
52576
52581
52582
52586
52607
52609
52617
52623
52627
52634
52636
52685
52730
52799
52814
52815
52830
52838
52853
52885
52915
52939
52942
52944
52957
52968
52977
52988
52992
53009
53015
53034
53035
53064
53073
53116
53141
53185
53186
53196
53205
53209
53213
53217
53228
53251
53275
53287
53295
53321
53322
53324
53325
53326
53335
53359
53366
53395
53397
53402
53405
53406
53411
53434
53442
53447
53458
53459
53484
53486
53498
53506
53509
53537
53554
53586
53611
53631
53661
53685
53719
53783
53796
53825
53840
53859
53878
53990
54008
54020
54072
54083
54087
54092
54119
54156
54165
54177
54178
5419

69477
69491
69492
69501
69503
69508
69510
69512
69545
69575
69576
69624
69627
69660
69681
69683
69688
69690
69710
69713
69740
69744
69758
69759
69820
69844
69853
69861
69866
69872
69881
69895
69896
69898
69905
69906
69907
69913
69916
69923
69931
69957
69978
69981
69983
70006
70024
70037
70047
70053
70056
70063
70074
70075
70080
70084
70107
70131
70140
70143
70145
70175
70184
70194
70206
70213
70225
70227
70254
70261
70266
70284
70285
70310
70324
70347
70425
70441
70496
70497
70516
70528
70558
70573
70594
70607
70627
70635
70638
70679
70690
70695
70698
70720
70722
70726
70728
70765
70771
70772
70788
70813
70817
70820
70846
70855
70862
70864
70866
70868
70875
70883
70930
70932
70949
70975
71004
71007
71018
71020
71042
71050
71052
71061
71069
71071
71072
71102
71121
71135
71151
71159
71198
71209
71210
71221
71224
71229
71280
71321
71326
71337
71343
71346
71350
71372
71400
71413
71414
71444
71458
71460
71463
71477
71505
71521
71522
71532
71534
71537
71557
71558
71574
71577
71579
71598
7159

In [271]:
over = over.append(pd.DataFrame({"node":list_node_1}),ignore_index=True,sort=False)
over.fillna(0)
over.head()

,node,下限,count
0,100012,1,1
1,100050,1,2
2,100110,2,15
3,100186,1,2
4,100216,1,1


In [273]:
over.to_excel("计算结果.xlsx",index=False)
print(len(over))

124042